In [1]:
!pip install kfp --upgrade

     |████████████████████████████████| 225 kB 3.0 MB/s eta 0:00:01
     |████████████████████████████████| 127 kB 4.9 MB/s eta 0:00:01
     |████████████████████████████████| 104 kB 10.9 MB/s eta 0:00:01
     |████████████████████████████████| 1.7 MB 4.2 MB/s eta 0:00:01
     |████████████████████████████████| 61 kB 537 bytes/s  0:00:01
     |████████████████████████████████| 54 kB 26 kB/s  eta 0:00:01
     |████████████████████████████████| 52 kB 24 kB/s  eta 0:00:011
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 87 kB 604 kB/s eta 0:00:01
     |████████████████████████████████| 95 kB 360 kB/s eta 0:00:011
     |████████████████████████████████| 75 kB 359 kB/s eta 0:00:011
     |████████████████████████████████| 151 kB 5.7 MB/s eta 0:00:01
  Created wheel for kfp: filename=kfp-1.6.4-py3-none-any.whl size=307978 sha256=5663f577907948dabfd44a6781dc72457da1954f4aa9e9d5

In [ ]:
from kfp.v2 import dsl
from kfp.v2.dsl import (Artifact,
                        Dataset,
                        Input,
                        Model,
                        Output,
                        Metrics,
                        component)

from kfp.v2 import compiler

from pipeline.preproc_split_dataset import preproc_split_dataset
from pipeline.train import train_evaluate
from pipeline.evaluate_model import evaluate_model
#from helper_components import build_image
#from helper_components import custom_deploy

In [ ]:
!gsutil mb gs -p PROJECT_ID -l REGION -b on gs://kubeflow-files
!gsutil ls

In [ ]:
REGION = 'us-central1'
PROJECT_ID = !(gcloud config get-value core/project)
PROJECT_ID = PROJECT_ID[0]

In [ ]:
@dsl.pipeline(
    # Default pipeline root. You can override it when submitting the pipeline.
    pipeline_root="gs://YOUR_BUCKET_NAME/SOME_FOLDER",
    # A name for the pipeline. Use to determine the pipeline Context.
    name="pipeline-test-1")


def pipeline():
    dataset_op = preproc_split_dataset()
    train_op = train_model(dataset_op.outputs["dataset_train"],alpha=0.001,max_iter=100)
    eval_op = evaluate_model(
        test_set=dataset_op.outputs["dataset_test"],
        model=train_op.outputs["model_artifact"]
    )

In [ ]:
compiler.Compiler().compile(pipeline_func=pipeline,
        package_path='sklearn_pipe.json')